# Load data and data analysis

In [ ]:
# TODO

# Information retrieval

In [ ]:
# TODO

# Setup generative model (open_model)

In [ ]:
from transformers import AutoTokenizer, OPTForCausalLM

open_model = OPTForCausalLM.from_pretrained("facebook/opt-1.3b")
open_tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [ ]:
prompt = """CONTEXT: 
Austria is a landlocked country of approximately 8.95 million inhabitants in Central Europe. It is bordered by the Czech Republic and Germany to the north, Slovakia and Hungary to the east, Slovenia and Italy to the south, and Switzerland and Liechtenstein to the west.
QUESTION:
Where is Austria?"""

inputs = open_tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = open_model.generate(inputs.input_ids, max_length=100)
answer = open_tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)

# Evaluation

In [ ]:
# TODO